In [1]:
cd drive/MyDrive/SimSong_base/SimSong

/content/drive/MyDrive/SimSong_base/SimSong


#### 다운로드된 파일 확인용

In [12]:
# 다운로드된 파일 개수 확인
import os
path = 'mp3'
file_list = os.listdir(path)
print(len(file_list))

2355


In [13]:
# 다운로드된 파일 총 용량 확인
import shutil
path = 'mp3'
total_size = 0
for file in os.listdir(path):
    file_path = os.path.join(path, file)
    total_size += os.path.getsize(file_path)

print(f"Total size: {total_size / (1024 * 1024):.2f} MB")

Total size: 21053.64 MB


#### mp3 다운로드

In [6]:
!pip install yt_dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.9/171.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 54.2 MB/s eta 0:00:00


In [7]:
import pandas as pd
import os
import yt_dlp
from yt_dlp import YoutubeDL
import time
import json

In [8]:
# Define file paths
base_file = 'data_v2/youtube_url.csv'
base_url = "https://www.youtube.com/watch?v="

# Read CSV files
df_base = pd.read_csv(base_file)

In [9]:
checkpoint_file = 'collect_data_v2/mp3_download_checkpoint.json'
# MP3 파일을 저장할 디렉토리
directory = 'mp3'

In [10]:
# 체크포인트 파일이 존재하면 읽고, 존재하지 않으면 0부터 시작
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'r') as f:
        try:
            checkpoint = json.load(f)
        except json.JSONDecodeError:
            # JSONDecodeError 발생 시 기본값으로 초기화
            checkpoint = {'last_processed_index': 0}
            with open(checkpoint_file, 'w') as f:
                json.dump(checkpoint, f)
    start_index = checkpoint.get('last_processed_index', 0)
else:
    start_index = 0

In [11]:
# base 파일에서 각 행을 순차적으로 처리, 체크포인트 인덱스 이후부터 시작
### ~2500개까지로 설정함

for index, row in df_base.iloc[start_index:2500].iterrows():
    # title과 artist를 합쳐서 검색할 문자열 생성
        # 일치하는 trackid 추출
        trackid = row["Track ID"]

        # yt-dlp의 출력 템플릿과 옵션 설정
        ydl_opts = {
            "format": 'bestaudio/best',
            'outtmpl': os.path.join(directory, f'{trackid}.%(ext)s'),  # trackid를 파일명으로 저장
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '320'
            }],
            'noplaylist': True  # 재생목록을 다운로드하지 않도록 설정

        }

        # yt-dlp를 이용해 오디오 다운로드
        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                ydl.download([base_url + row["video_id"]])

            # 다운로드가 성공적으로 완료되면 체크포인트 업데이트
            checkpoint = {'last_processed_index': index + 1}  # 다음 곡의 인덱스를 저장
            with open(checkpoint_file, 'w') as f:
                json.dump(checkpoint, f)

            print(f"{row['title']} by {row['artist']} 다운로드 완료")

        except Exception as e:
            print(f"{row['title']} by {row['artist']} 다운로드 중 오류 발생: {e}")
            # 오류 발생 시 계속해서 다음 곡을 처리하려면 continue 사용
            continue


# 모든 다운로드가 완료되면 체크포인트 파일을 삭제할 수 있음 (선택 사항)
# os.remove(checkpoint_file)

[youtube] Extracting URL: https://www.youtube.com/watch?v=ZFcdqtHHMVs
[youtube] ZFcdqtHHMVs: Downloading webpage
[youtube] ZFcdqtHHMVs: Downloading tv client config
[youtube] ZFcdqtHHMVs: Downloading player c548b3da
[youtube] ZFcdqtHHMVs: Downloading tv player API JSON
[youtube] ZFcdqtHHMVs: Downloading ios player API JSON
[youtube] ZFcdqtHHMVs: Downloading m3u8 information
[info] ZFcdqtHHMVs: Downloading 1 format(s): 251
[download] Destination: mp3/TRWHCCQ128F42515E4.webm
[download] 100% of    3.77MiB in 00:00:00 at 7.65MiB/s   
[ExtractAudio] Destination: mp3/TRWHCCQ128F42515E4.mp3
Deleting original file mp3/TRWHCCQ128F42515E4.webm (pass -k to keep)
What's The Matter by Gregory Isaacs 다운로드 완료
[youtube] Extracting URL: https://www.youtube.com/watch?v=Ke915BsiPDw
[youtube] Ke915BsiPDw: Downloading webpage
[youtube] Ke915BsiPDw: Downloading tv client config
[youtube] Ke915BsiPDw: Downloading player c548b3da
[youtube] Ke915BsiPDw: Downloading tv player API JSON
[youtube] Ke915BsiPDw: Dow

ERROR: [youtube] lHQ0va_ZEvY: Video unavailable. This video is not available


Loving That You Want by Pinchers 다운로드 중 오류 발생: ERROR: [youtube] lHQ0va_ZEvY: Video unavailable. This video is not available
[youtube] Extracting URL: https://www.youtube.com/watch?v=8UvQvOFRS9o
[youtube] 8UvQvOFRS9o: Downloading webpage
[youtube] 8UvQvOFRS9o: Downloading tv client config
[youtube] 8UvQvOFRS9o: Downloading player c548b3da
[youtube] 8UvQvOFRS9o: Downloading tv player API JSON
[youtube] 8UvQvOFRS9o: Downloading ios player API JSON


ERROR: [youtube] 8UvQvOFRS9o: Video unavailable. The uploader has not made this video available in your country


Hands Around My Throat by Death In Vegas 다운로드 중 오류 발생: ERROR: [youtube] 8UvQvOFRS9o: Video unavailable. The uploader has not made this video available in your country
[youtube] Extracting URL: https://www.youtube.com/watch?v=aBMh-mvE3c0
[youtube] aBMh-mvE3c0: Downloading webpage
[youtube] aBMh-mvE3c0: Downloading tv client config
[youtube] aBMh-mvE3c0: Downloading player 56511309
[youtube] aBMh-mvE3c0: Downloading tv player API JSON
[youtube] aBMh-mvE3c0: Downloading ios player API JSON
[youtube] aBMh-mvE3c0: Downloading m3u8 information
[info] aBMh-mvE3c0: Downloading 1 format(s): 140
[download] Destination: mp3/TRWAFEE128F4254ED9.m4a
[download] 100% of    4.68MiB in 00:00:00 at 10.03MiB/s  
[FixupM4a] Correcting container of "mp3/TRWAFEE128F4254ED9.m4a"
[ExtractAudio] Destination: mp3/TRWAFEE128F4254ED9.mp3
Deleting original file mp3/TRWAFEE128F4254ED9.m4a (pass -k to keep)
Anal Chinook by Dayglo Abortions 다운로드 완료
[youtube] Extracting URL: https://www.youtube.com/watch?v=GVTWLDh47M8


...

Like A River by Kasey Chambers 다운로드 중 오류 발생: ERROR: [youtube] 2te8cMXUeos: Video unavailable. The uploader has not made this video available in your country
[youtube] Extracting URL: https://www.youtube.com/watch?v=QyZfQZYHISY
[youtube] QyZfQZYHISY: Downloading webpage
[youtube] QyZfQZYHISY: Downloading tv client config
[youtube] QyZfQZYHISY: Downloading player c548b3da
[youtube] QyZfQZYHISY: Downloading tv player API JSON
[youtube] QyZfQZYHISY: Downloading ios player API JSON
[youtube] QyZfQZYHISY: Downloading m3u8 information
[info] QyZfQZYHISY: Downloading 1 format(s): 251
[download] Destination: mp3/TRMAVCJ128F4287DE0.webm
[download] 100% of    3.63MiB in 00:00:01 at 3.41MiB/s   
[ExtractAudio] Destination: mp3/TRMAVCJ128F4287DE0.mp3
Deleting original file mp3/TRMAVCJ128F4287DE0.webm (pass -k to keep)
When The Moon Comes Out Tonight by Alastair Moock 다운로드 완료
[youtube] Extracting URL: https://www.youtube.com/watch?v=oCEglv71vrU
[youtube] oCEglv71vrU: Downloading webpage
[youtube] oC

ERROR: [youtube] 9N1mEIR9ARo: Video unavailable. The uploader has not made this video available in your country


Underwear by Loudon Wainwright III 다운로드 중 오류 발생: ERROR: [youtube] 9N1mEIR9ARo: Video unavailable. The uploader has not made this video available in your country
[youtube] Extracting URL: https://www.youtube.com/watch?v=TLqHIrxSJZY
[youtube] TLqHIrxSJZY: Downloading webpage
[youtube] TLqHIrxSJZY: Downloading tv client config
[youtube] TLqHIrxSJZY: Downloading player c548b3da
[youtube] TLqHIrxSJZY: Downloading tv player API JSON
[youtube] TLqHIrxSJZY: Downloading ios player API JSON
[youtube] TLqHIrxSJZY: Downloading m3u8 information
[info] TLqHIrxSJZY: Downloading 1 format(s): 251
[download] Destination: mp3/TRGTJWQ128F1469950.webm
[download] 100% of    2.88MiB in 00:00:00 at 5.69MiB/s   
[ExtractAudio] Destination: mp3/TRGTJWQ128F1469950.mp3
Deleting original file mp3/TRGTJWQ128F1469950.webm (pass -k to keep)
OGM by Loudon Wainwright III 다운로드 완료
[youtube] Extracting URL: https://www.youtube.com/watch?v=kc4EWTgx81w
[youtube] kc4EWTgx81w: Downloading webpage
[youtube] kc4EWTgx81w: Downlo

ERROR: unable to download video data: HTTP Error 403: Forbidden


Better Days (LP Version) by Guy Clark 다운로드 중 오류 발생: ERROR: unable to download video data: HTTP Error 403: Forbidden
[youtube] Extracting URL: https://www.youtube.com/watch?v=mwLE81jABjE
[youtube] mwLE81jABjE: Downloading webpage
[youtube] mwLE81jABjE: Downloading tv client config
[youtube] mwLE81jABjE: Downloading player c548b3da
[youtube] mwLE81jABjE: Downloading tv player API JSON
[youtube] mwLE81jABjE: Downloading ios player API JSON
[youtube] mwLE81jABjE: Downloading m3u8 information
[info] mwLE81jABjE: Downloading 1 format(s): 140
[download] Destination: mp3/TRHUFTC128F428551A.m4a
[download] 100% of    3.97MiB in 00:00:00 at 9.60MiB/s   
[FixupM4a] Correcting container of "mp3/TRHUFTC128F428551A.m4a"
[ExtractAudio] Destination: mp3/TRHUFTC128F428551A.mp3
Deleting original file mp3/TRHUFTC128F428551A.m4a (pass -k to keep)
Texas Cookin' by Guy Clark 다운로드 완료
[youtube] Extracting URL: https://www.youtube.com/watch?v=76hpKJKE5DA
[youtube] 76hpKJKE5DA: Downloading webpage
[youtube] 76hp